In [56]:
#importing dependencies
import pandas as pd
import numpy as np
import functools
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier

In [42]:
#importing our data and specify data types 
df = pd.read_csv("ecommerce.csv",encoding="ISO-8859-1",dtype={'CustomerID': str,'InvoiceID': str})

In [43]:
df.InvoiceDate = pd.to_datetime(df.InvoiceDate, format="%m/%d/%Y %H:%M")
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null object
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 33.1+ MB


In [44]:
#preview our data
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [45]:
#remove the negative values and replace with nan
df[df['Quantity'] < 0] = np.nan
df[df['UnitPrice'] < 0] = np.nan
df.describe()

,Quantity,UnitPrice
count,531283.000000,531283.000000
mean,10.655299,3.898954
std,156.830617,35.876279
min,1.000000,0.000000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,13541.330000


In [46]:
#get the total spent for each line item
#adding total_dollars to data frame 
df['total_dollars'] = df['Quantity']*df['UnitPrice']
#describe allows us to see count, mean, standard deviation, quartiles, min and max
df.describe()

,Quantity,UnitPrice,total_dollars
count,531283.000000,531283.000000,531283.000000
mean,10.655299,3.898954,20.077218
std,156.830617,35.876279,270.058254
min,1.000000,0.000000,0.000000
25%,1.000000,1.250000,3.750000
50%,3.000000,2.080000,9.900000
75%,10.000000,4.130000,17.700000
max,80995.000000,13541.330000,168469.600000


In [47]:
#how many orders have they made
#as_index = FASLE because we do not want the column to be indexed later on
invoice_ct = df.groupby(by='CustomerID', as_index=False)['InvoiceNo'].count()
invoice_ct.columns = ['CustomerID', 'NumberOrders']
invoice_ct.describe()

,NumberOrders
count,4339.000000
mean,91.708689
std,228.792852
min,1.000000
25%,17.000000
50%,41.000000
75%,100.000000
max,7847.000000


In [48]:
#add up how much money have the customers have spent
total_spend = df.groupby(by='CustomerID', as_index=False)['total_dollars'].sum()
total_spend.columns = ['CustomerID', 'total_spent']
#snapshot of data
total_spend.describe()

,total_spent
count,4339.000000
mean,2053.793018
std,8988.248381
min,0.000000
25%,307.245000
50%,674.450000
75%,1661.640000
max,280206.020000


In [49]:
#add up how many items the customers have bought
total_items = df.groupby(by='CustomerID', as_index=False)['Quantity'].sum()
total_items.columns = ['CustomerID', 'NumberItems']
#snapshot of data
total_items.describe()

,NumberItems
count,4339.000000
mean,1194.214335
std,5055.603049
min,1.000000
25%,160.000000
50%,379.000000
75%,993.000000
max,197491.000000


In [50]:
#when was their first order and how long ago was that from the last date in file (presumably
#when the data were pulled)
earliest_order = df.groupby(by='CustomerID', as_index=False)['InvoiceDate'].min()
earliest_order.columns = ['CustomerID', 'EarliestInvoice']
earliest_order['now'] = pd.to_datetime((df['InvoiceDate']).max())
earliest_order['days_as_customer'] = 1 + (earliest_order.now-earliest_order.EarliestInvoice).astype('timedelta64[D]')
earliest_order.drop('now', axis=1, inplace=True)
#using head to only show most recent data
earliest_order.head()

,CustomerID,EarliestInvoice,days_as_customer
0,12346,2011-01-18 10:01:00,326.0
1,12347,2010-12-07 14:57:00,367.0
2,12348,2010-12-16 19:09:00,358.0
3,12349,2011-11-21 09:51:00,19.0
4,12350,2011-02-02 16:01:00,310.0


In [51]:
#when was their last order and how long ago was that from the last date in file (presumably
#when the data were pulled)
last_order = df.groupby(by='CustomerID', as_index=False)['InvoiceDate'].max()
last_order.columns = ['CustomerID', 'last_purchase']
last_order['now'] = pd.to_datetime((df['InvoiceDate']).max())
last_order['days_since_purchase'] = 1 + (last_order.now-last_order.last_purchase).astype('timedelta64[D]')
last_order.drop('now', axis=1, inplace=True)
#using head to only show most recent data
last_order.head()

,CustomerID,last_purchase,days_since_purchase
0,12346,2011-01-18 10:01:00,326.0
1,12347,2011-12-07 15:52:00,2.0
2,12348,2011-09-25 13:13:00,75.0
3,12349,2011-11-21 09:51:00,19.0
4,12350,2011-02-02 16:01:00,310.0


In [52]:
#combining all of the dataframes into one
dfs = [total_spend,invoice_ct,earliest_order,last_order,total_items]
CustomerTable = functools.reduce(lambda left,right: pd.merge(left,right,on='CustomerID', how='outer'), dfs)
CustomerTable.head()

,CustomerID,total_spent,NumberOrders,EarliestInvoice,days_as_customer,last_purchase,days_since_purchase,NumberItems
0,12346,77183.60,1,2011-01-18 10:01:00,326.0,2011-01-18 10:01:00,326.0,74215.0
1,12347,4310.00,182,2010-12-07 14:57:00,367.0,2011-12-07 15:52:00,2.0,2458.0
2,12348,1797.24,31,2010-12-16 19:09:00,358.0,2011-09-25 13:13:00,75.0,2341.0
3,12349,1757.55,73,2011-11-21 09:51:00,19.0,2011-11-21 09:51:00,19.0,631.0
4,12350,334.40,17,2011-02-02 16:01:00,310.0,2011-02-02 16:01:00,310.0,197.0


In [53]:
#customer count is an important metric and we use the following command to see the number of customers
len(CustomerTable)

4339

In [54]:
#identify and separate big spenders, lots of orders, long-time customers, dormant customers for
#sales and marketing campaign use; need to be separate flags because they aren't all mutually
#exclusive

def big_spender(row):
    if row['total_spent'] >= 1661.64:
        return 'Yes'
    else:
        return 'No'

def many_orders(row):
    if row['NumberOrders'] >= 100:
        return 'Yes'
    else:
        return 'No'

def loyal_customer(row):
    if row['days_as_customer'] >= 326:
        return 'Yes' 
    else:
        return 'No'

def dormant_customer(row):
    if row['days_since_purchase'] >= 141:
        return 'Yes' 
    else:
        return 'No'

CustomerTable['BigSpender'] = CustomerTable.apply(big_spender, axis=1)
CustomerTable['ManyOrders'] = CustomerTable.apply(many_orders, axis=1)
CustomerTable['LoyalCustomer'] = CustomerTable.apply(loyal_customer, axis=1)
CustomerTable['DormantCustomer'] = CustomerTable.apply(dormant_customer, axis=1)
#simple arithmetic
CustomerTable['OrderFrequency'] = CustomerTable['NumberOrders']/CustomerTable['days_as_customer']

CustomerTable.head()

,CustomerID,total_spent,NumberOrders,EarliestInvoice,days_as_customer,last_purchase,days_since_purchase,NumberItems,BigSpender,ManyOrders,LoyalCustomer,DormantCustomer,OrderFrequency
0,12346,77183.60,1,2011-01-18 10:01:00,326.0,2011-01-18 10:01:00,326.0,74215.0,Yes,No,Yes,Yes,0.003067
1,12347,4310.00,182,2010-12-07 14:57:00,367.0,2011-12-07 15:52:00,2.0,2458.0,Yes,Yes,Yes,No,0.495913
2,12348,1797.24,31,2010-12-16 19:09:00,358.0,2011-09-25 13:13:00,75.0,2341.0,Yes,No,Yes,No,0.086592
3,12349,1757.55,73,2011-11-21 09:51:00,19.0,2011-11-21 09:51:00,19.0,631.0,Yes,No,No,No,3.842105
4,12350,334.40,17,2011-02-02 16:01:00,310.0,2011-02-02 16:01:00,310.0,197.0,No,No,No,Yes,0.054839


In [59]:
#look at the distributions and relationships with other continuous variables
import seaborn as sns
sns.pairplot(CustomerTable, vars=["total_spent", "NumberOrders",'days_as_customer',
                                  'days_since_purchase','NumberItems','OrderFrequency'])

In [60]:
RF = CustomerTable[["NumberOrders",'days_as_customer','NumberItems','BigSpender','CustomerID']]
features = RF.columns[:3]
features

Index(['NumberOrders', 'days_as_customer', 'NumberItems'], dtype='object')

In [81]:
RF['is_train'] = np.random.uniform(0, 1, len(RF)) <= .8
train, test = RF[RF['is_train']==True], RF[RF['is_train']==False]
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 3484
Number of observations in the test data: 855


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [82]:
y = pd.factorize(train['BigSpender'])[0]
y[0:10] #show the first ten; 'No' = 0

array([0, 0, 0, 1, 0, 1, 1, 1, 0, 0])

In [83]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=2)
clf.fit(train[features], y)
list(zip(train[features], clf.feature_importances_))

[('NumberOrders', 0.22434207548173557),
 ('days_as_customer', 0.13794989630424545),
 ('NumberItems', 0.63770802821401895)]

In [84]:
clf.predict_proba(test[features])[0:10]


array([[ 1. ,  0. ],
       [ 0.6,  0.4],
       [ 0.6,  0.4],
       [ 0.8,  0.2],
       [ 0.2,  0.8],
       [ 0. ,  1. ],
       [ 0. ,  1. ],
       [ 0. ,  1. ],
       [ 0.9,  0.1],
       [ 0. ,  1. ]])

In [85]:
test['Prediction'] = clf.predict(test[features])
test.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,NumberOrders,days_as_customer,NumberItems,BigSpender,CustomerID,is_train,Prediction
1,182,367.0,2458.0,Yes,12347,False,0
13,129,201.0,1165.0,Yes,12360,False,0
26,77,355.0,944.0,No,12377,False,0
31,99,352.0,1532.0,Yes,12383,False,0
37,64,333.0,816.0,No,12393,False,1
